# 네이버 cloud flatform에서 지도 API 제공

In [1]:
import folium
import pandas as pd
import urllib.request
import datetime
import time
import json
from config_NaverAPI_okcy import *    # NAVER Client ID/Secret

In [2]:
import os
import sys
import webbrowser

## 네이버 지도 API-id, secret-key

In [3]:
#[CODE 1]
def get_request_url(url):
    
    req = urllib.request.Request(url)
    req.add_header("X-NCP-APIGW-API-KEY-ID", client_id)
    req.add_header("X-NCP-APIGW-API-KEY", client_secret)
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [4]:
base = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
address = "울산시 남구 대학로 93"
parameters = "?query=%s" % urllib.parse.quote(address)
url = base + parameters
retData = get_request_url(url)

[2022-12-13 23:59:04.168342] Url Request Success


In [5]:
jsonAddress = json.loads(retData)
jsonAddress

{'status': 'OK',
 'meta': {'totalCount': 1, 'page': 1, 'count': 1},
 'addresses': [{'roadAddress': '울산광역시 남구 대학로 93 울산대학교',
   'jibunAddress': '울산광역시 남구 무거동 665 울산대학교',
   'englishAddress': '93, Daehak-ro, Nam-gu, Ulsan, Republic of Korea',
   'addressElements': [{'types': ['SIDO'],
     'longName': '울산광역시',
     'shortName': '울산광역시',
     'code': ''},
    {'types': ['SIGUGUN'], 'longName': '남구', 'shortName': '남구', 'code': ''},
    {'types': ['DONGMYUN'], 'longName': '무거동', 'shortName': '무거동', 'code': ''},
    {'types': ['RI'], 'longName': '', 'shortName': '', 'code': ''},
    {'types': ['ROAD_NAME'],
     'longName': '대학로',
     'shortName': '대학로',
     'code': ''},
    {'types': ['BUILDING_NUMBER'],
     'longName': '93',
     'shortName': '93',
     'code': ''},
    {'types': ['BUILDING_NAME'],
     'longName': '울산대학교',
     'shortName': '울산대학교',
     'code': ''},
    {'types': ['LAND_NUMBER'],
     'longName': '665',
     'shortName': '665',
     'code': ''},
    {'types': ['POSTAL

In [6]:
longitude = jsonAddress['addresses'][0]['x']
latitude = jsonAddress['addresses'][0]['y']
print([latitude, longitude])

['35.5437482', '129.2561333']


## 검색 주소의 지도 정보를JSON 형식으로 return

In [7]:
#[CODE 2]
def getGeoData(address):
    
    base = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
    
    try:
        parameters = "?query=%s" % urllib.parse.quote(address)
    except:
        return None
    
    url = base + parameters
    
    retData = get_request_url(url)
    if retData == None:
        return None

    jsonAddress = json.loads(retData)

    if 'addresses' in jsonAddress.keys():
        if jsonAddress['addresses'] != []:
            latitude = jsonAddress['addresses'][0]['y']
            longitude = jsonAddress['addresses'][0]['x']
        else:
            return None
    else:
        return None
    
    return [latitude, longitude]

## folium map 객체 생성(울산대학교 )

In [8]:
map_uou = folium.Map(location=[latitude,longitude], zoom_start=12)
map_uou

In [9]:
folium.Marker([35.5437411,129.2540956], popup='울산대학교').add_to(map_uou)
map_uou

## 가맹점 데이터 파일에서 주소

In [10]:
import pdb
filename = 'chicken_data/cheogajip_table.csv'
df = pd.read_csv(filename, encoding='CP949', index_col=0, header=0)
geoData = []
df

,store,sido,gungu,store_address
0,울산대점,울산광역시,남구,울산광역시 남구 대학로 94번길 6
1,수암점,울산광역시,남구,울산광역시 남구 중앙로 38(야음동)
2,상록점,경기도,안산시,경기도 안산시 상록구 서암로4길 20-1 (사동)
3,죽전점,경기도,용인수,경기도 용인수 수지구 현암로76 (죽전동)
4,괴정샘터점,부산광역시,사하구,부산광역시 사하구 낙동대로 233번길 59
...,...,...,...,...
1045,소라점,충청북도,청주시,충청북도 청주시 상당구 원봉로 25 신라아파트 상가 111호
1046,충북덕산점,충청북도,진천군,충청북도 진천군 덕산면 초금로 757-2
1047,영동점,충청북도,영동군,충청북도 영동군 영동읍 영산로1길 3
1048,옥천점,충청북도,옥천군,충청북도 옥천군 옥천읍 금장로 113


In [11]:
#[CODE 4]
import pdb
filename = 'chicken_data/cheogajip_table.csv'
df = pd.read_csv(filename, encoding='CP949', index_col=0, header=0)
geoData = []

for index, row in df.iterrows():
    if row['sido'] == '울산광역시':
        geoData = getGeoData(row['store_address'])
        if geoData != None:
            folium.Marker(geoData, popup=row['store'], icon=folium.Icon(color='red')).add_to(map_chicken)

[2022-12-13 23:59:49.307534] Url Request Success


NameError: name 'map_chicken' is not defined

In [40]:
map_chicken

In [13]:
map_chicken = folium.Map(location=[35.5437411,129.2540956], zoom_start=12)
map_chicken

In [41]:
filename = 'chicken_data/kyochon_table.csv'
df = pd.read_csv(filename, encoding='CP949', index_col=0, header=0)
geoData = []
for index, row in df.iterrows():
    if row['sido'] == '울산광역시':
        geoData = getGeoData(row['store_address'])
        if geoData != None:
            folium.Marker(geoData, popup=row['store'], icon=folium.Icon(color='blue')).add_to(map_chicken)

[2021-12-07 17:20:59.176991] Url Request Success
[2021-12-07 17:20:59.270657] Url Request Success
[2021-12-07 17:20:59.374617] Url Request Success
[2021-12-07 17:20:59.480661] Url Request Success
[2021-12-07 17:20:59.583007] Url Request Success
[2021-12-07 17:20:59.684914] Url Request Success
[2021-12-07 17:20:59.783483] Url Request Success
[2021-12-07 17:20:59.879120] Url Request Success
[2021-12-07 17:20:59.980006] Url Request Success
[2021-12-07 17:21:00.085006] Url Request Success
[2021-12-07 17:21:00.182007] Url Request Success
[2021-12-07 17:21:00.291006] Url Request Success
[2021-12-07 17:21:00.390007] Url Request Success
[2021-12-07 17:21:00.484007] Url Request Success
[2021-12-07 17:21:00.579005] Url Request Success
[2021-12-07 17:21:00.692006] Url Request Success
[2021-12-07 17:21:00.794008] Url Request Success
[2021-12-07 17:21:00.891006] Url Request Success
[2021-12-07 17:21:00.985008] Url Request Success
[2021-12-07 17:21:01.092005] Url Request Success
[2021-12-07 17:21:01

In [42]:
map_chicken

In [43]:
filename = 'chicken_data/goobne_table.csv'
df = pd.read_csv(filename, encoding='CP949', index_col=0, header=0)
geoData = []
for index, row in df.iterrows():
    if row['sido'] == '울산광역시':
        geoData = getGeoData(row['store_address'])
        if geoData != None:
            folium.Marker(geoData, popup=row['store'], icon=folium.Icon(color='gray')).add_to(map_chicken)

[2021-12-07 17:21:30.817125] Url Request Success
[2021-12-07 17:21:30.920124] Url Request Success
[2021-12-07 17:21:31.016126] Url Request Success
[2021-12-07 17:21:31.134125] Url Request Success
[2021-12-07 17:21:31.234125] Url Request Success
[2021-12-07 17:21:31.328125] Url Request Success
[2021-12-07 17:21:31.438128] Url Request Success
[2021-12-07 17:21:31.543125] Url Request Success
[2021-12-07 17:21:31.639126] Url Request Success
[2021-12-07 17:21:31.742127] Url Request Success
[2021-12-07 17:21:31.865126] Url Request Success
[2021-12-07 17:21:31.960173] Url Request Success
[2021-12-07 17:21:32.066180] Url Request Success
[2021-12-07 17:21:32.162174] Url Request Success
[2021-12-07 17:21:32.290175] Url Request Success
[2021-12-07 17:21:32.388173] Url Request Success
[2021-12-07 17:21:32.485174] Url Request Success
[2021-12-07 17:21:32.600173] Url Request Success
[2021-12-07 17:21:32.707173] Url Request Success
[2021-12-07 17:21:32.903232] Url Request Success
[2021-12-07 17:21:33

In [44]:
map_chicken

In [47]:
filename = 'chicken_data/nene_table.csv'
df = pd.read_csv(filename, encoding='CP949', index_col=0, header=0)
geoData = []
for index, row in df.iterrows():
    if row['sido'] == '울산광역시':
        geoData = getGeoData(row['store_address'])
        if geoData != None:
            folium.Marker(geoData, popup=row['store'], icon=folium.Icon(color='pink')).add_to(map_chicken)

[2021-12-07 17:22:42.735577] Url Request Success
[2021-12-07 17:22:42.830171] Url Request Success
[2021-12-07 17:22:42.925187] Url Request Success
[2021-12-07 17:22:43.036344] Url Request Success
[2021-12-07 17:22:43.135343] Url Request Success
[2021-12-07 17:22:43.236343] Url Request Success
[2021-12-07 17:22:43.335662] Url Request Success
[2021-12-07 17:22:43.432689] Url Request Success
[2021-12-07 17:22:43.530783] Url Request Success
[2021-12-07 17:22:43.653555] Url Request Success
[2021-12-07 17:22:43.748554] Url Request Success
[2021-12-07 17:22:43.841489] Url Request Success
[2021-12-07 17:22:43.937780] Url Request Success
[2021-12-07 17:22:44.039703] Url Request Success
[2021-12-07 17:22:44.132704] Url Request Success
[2021-12-07 17:22:44.233704] Url Request Success
[2021-12-07 17:22:44.327641] Url Request Success
[2021-12-07 17:22:44.436641] Url Request Success
[2021-12-07 17:22:44.532639] Url Request Success
[2021-12-07 17:22:44.632640] Url Request Success
[2021-12-07 17:22:44

In [48]:
map_chicken

In [24]:
map_chicken = folium.Map(location=[35.5437411,129.2540956], zoom_start=12)
map_chicken

In [53]:
filename = 'chicken_data/pericana_table.csv'
df = pd.read_csv(filename, encoding='CP949', index_col=0, header=0)
geoData = []
for index, row in df.iterrows():
    if row['sido'] == '울산광역시':
        geoData = getGeoData(row['store_address'])
        if geoData != None:
            folium.Marker(geoData, popup=row['store'], icon=folium.Icon(color='black')).add_to(map_chicken)

[2021-12-07 17:23:58.163232] Url Request Success
[2021-12-07 17:23:58.257234] Url Request Success
[2021-12-07 17:23:58.352230] Url Request Success
[2021-12-07 17:23:58.459233] Url Request Success
[2021-12-07 17:23:58.567231] Url Request Success
[2021-12-07 17:23:58.673230] Url Request Success
[2021-12-07 17:23:58.795231] Url Request Success
[2021-12-07 17:23:58.895233] Url Request Success
[2021-12-07 17:23:59.001231] Url Request Success
[2021-12-07 17:23:59.161232] Url Request Success
[2021-12-07 17:23:59.260230] Url Request Success
[2021-12-07 17:23:59.379231] Url Request Success
[2021-12-07 17:23:59.485233] Url Request Success
[2021-12-07 17:23:59.604231] Url Request Success
[2021-12-07 17:23:59.706233] Url Request Success
[2021-12-07 17:23:59.809230] Url Request Success
[2021-12-07 17:23:59.927231] Url Request Success
[2021-12-07 17:24:00.023906] Url Request Success
[2021-12-07 17:24:00.122086] Url Request Success
[2021-12-07 17:24:00.248112] Url Request Success


In [54]:
map_chicken

In [55]:
#[CODE 5]
svFilename = 'chicken_가맹점_울산광역시.html'
map_chicken.save(svFilename)
webbrowser.open(svFilename)

True